# CleanEx: Model selection
In this notebook, I am using recursive feature elimination to perform model selection in the training data.

In [1]:
# load entire classification dataset (n=3400 genes)
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# sample top and bottom 10% and classify
d = pd.read_csv(open("input_data_top-bottom_classify.csv"), index_col=0)
# remove unncessary columns
d = d.drop(['liver.ratio', 'log2.liver.ratio', 'decile'], 1)


In [2]:
# randomly split data into training and test sets (70:30 split), normalizing the predictors
# set random number seed to be replicable
seed = np.random.seed(seed=6892)
# randomly assign the order
d['random_order'] = np.random.permutation(len(d))
d = d.sort_values(by='random_order', ascending=True)
d = d.drop('random_order', 1)

# separate outcome from predictors
X = d.drop('highly.liver.expressed', 1)
y = d['highly.liver.expressed']

# normalize predictors
X = (X-np.mean(X, axis=0))/np.std(X, axis=0)

# split:
X_train = X[:-1000]
X_test  = X[-1000:]
y_train = y[:-1000]
y_test  = y[-1000:]


## Saturated model
Here I am running the saturated model to ensure that the features can reliably classify the top 10% and bottom 10% of genes in the training data, using 5-fold cross-validation.

In [3]:
# Logistic regression and print the mean and SD of the accuracy across all 5-folds
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
my_logreg = LogisticRegression()
scores = cross_val_score(my_logreg, X=X_train, y=y_train, cv=5)
print np.mean(scores), np.std(scores)


0.55994403656 0.0179865029108


In [90]:
# Support Vector Machines and print the mean and SD of the accuracy across all 5-folds
from sklearn.svm import SVC
my_svc = SVC(kernel="linear")
scores = cross_val_score(my_svc, X=X_train, y=y_train, cv=5)
print np.mean(scores), np.std(scores)


0.553655605554 0.015572048752


In [5]:
# Random Forests and print the mean and SD of the accuracy across all 5-folds
from sklearn.ensemble import RandomForestClassifier
my_rfc = RandomForestClassifier(random_state=2)
scores = cross_val_score(my_rfc, X=X_train, y=y_train, cv=5)
print np.mean(scores), np.std(scores)


0.563711481277 0.0203670459472


## Recursive Feature Elimination
Here I am using recursive feature elimination to eliminate superfluous features.

In [37]:
# recursive feature elimination for logistic regression
from sklearn.feature_selection import RFE
selector = RFE(my_logreg, 200, step=1000)
selector = selector.fit(X=X_train, y=y_train)
#print selector.ranking_

# scores of log reg after RFE:
r = []
for i in [200, 20, 10]:
    selector = RFE(my_logreg, i, step=1000)
    scores = cross_val_score(selector, X=X_train, y=y_train, cv=5)
    r.append([i, np.mean(scores), np.std(scores)])
print pd.DataFrame(r)

     0         1         2
0  200  0.546113  0.014986
1   20  0.551566  0.028200
2   10  0.529765  0.031236


In [91]:
# recursive feature elimination for support vector machines
r = []
for i in [200, 20, 10]:
    selector = RFE(my_svc, i, step=1000)
    scores = cross_val_score(selector, X=X_train, y=y_train, cv=5)
    r.append([i, np.mean(scores), np.std(scores)])
print pd.DataFrame(r)

     0         1         2
0  200  0.541074  0.014655
1   20  0.542744  0.013471
2   10  0.525141  0.010749


In [6]:
# recursive feature elimination for random forests
from sklearn.feature_selection import RFE
r = []
for i in [200, 20, 10]:
    selector = RFE(my_rfc, i, step=1000)
    scores = cross_val_score(selector, X=X_train, y=y_train, cv=5)
    r.append([i, np.mean(scores), np.std(scores)])
print pd.DataFrame(r)

     0         1         2
0  200  0.567051  0.010256
1   20  0.576267  0.019045
2   10  0.557012  0.017123


In [7]:
# print 20 most important features selected in random forests model
selector = RFE(my_rfc, 20, step=1000)
selector = selector.fit(X=X_train, y=y_train)

headers = np.array(list(X_train))
print headers[selector.support_]


['AC' 'AGG' 'AGGG' 'CC' 'CG' 'CGAA' 'CT' 'CTG' 'TCA' 'TCT' 'TGG' 'MA0528.1'
 'MA0528.1_max.score' 'MA0685.1_max.score' 'MA0056.1_max.start.pos'
 'MA0149.1_max.start.pos' 'MA0478.1_max.start.pos' 'MA0508.1_max.start.pos'
 'MA0597.1_max.start.pos' 'MA0759.1_max.start.pos']


In [112]:
# print vector of column indices of 20 most important features
s = selector.ranking_
i, = np.where( s==1 )
print i

[ 342  854  897 1707 2048 2050 2389 2560 4438 4693 4949 5593 5979 6040 6254
 6304 6334 6353 6369 6492]


In [8]:
# perform a random forests on these features
X_select = X_train.loc[:, headers[selector.support_]]

my_rfc = RandomForestClassifier(random_state=2)
my_rfc.fit(X=X_select, y=y_train)

# get feature importances
a = [headers[selector.support_]]
a.append(my_rfc.feature_importances_)
a = pd.DataFrame(a).transpose()
a.to_csv("RF_top20features.csv")
print a.sort_values(by=1, ascending=False)

# rank features by importance


                         0          1
4                       CG  0.0734069
12      MA0528.1_max.score  0.0704381
3                       CC  0.0653915
14  MA0056.1_max.start.pos  0.0577468
9                      TCT  0.0574966
6                       CT  0.0567954
8                      TCA  0.0564194
7                      CTG  0.0557282
10                     TGG  0.0554651
0                       AC  0.0550295
15  MA0149.1_max.start.pos  0.0547808
1                      AGG  0.0540682
2                     AGGG  0.0489609
17  MA0508.1_max.start.pos  0.0456348
11                MA0528.1  0.0451924
13      MA0685.1_max.score  0.0365594
16  MA0478.1_max.start.pos  0.0360116
5                     CGAA  0.0281987
18  MA0597.1_max.start.pos  0.0255982
19  MA0759.1_max.start.pos  0.0210776


## Check model accuracy in test data
Here I am carrying forward the 20 features identified in the random forest model and assessing the prediction accuracy of the 3 different models on the test data.

In [10]:
# predict test data with random forests and check accuracy
X_test_select = X_test.loc[:, headers[selector.support_]]
my_rfc.score(X=X_test_select, 
                y=y_test)

0.55700000000000005

In [126]:
# predict test data with support vector machines and check accuracy
my_svc = SVC(kernel="linear")
my_svc.fit(X=X_select, y=y_train)
my_svc.score(X=X_test_select, y=y_test)


0.59099999999999997

In [11]:
# predict test data with logistic regression and check accuracy
my_logreg = LogisticRegression()
my_logreg.fit(X=X_select, y=y_train)


my_logreg.score(X=X_test_select, 
                y=y_test)


0.60199999999999998

In [81]:
# print coefficients of logistic regression (obtain direction of effect)
b = [np.array(list(X_select))]
b.append(my_logreg.coef_.reshape(20,))
b = pd.DataFrame(b).transpose()
b.to_csv("top20features_LogReg_coefficients.csv")
print b

                         0          1
0                       AC   0.170571
1                      AGG -0.0465778
2                     AGGG  -0.107238
3                       CC   -0.13875
4                       CG   0.332335
5                     CGAA    0.10882
6                       CT  0.0419476
7                      CTG -0.0752032
8                      TCA   0.134025
9                      TCT -0.0747132
10                     TGG  0.0944133
11                MA0528.1 -0.0535979
12      MA0528.1_max.score   -0.14182
13      MA0685.1_max.score   0.173017
14  MA0056.1_max.start.pos -0.0435735
15  MA0149.1_max.start.pos  -0.147209
16  MA0478.1_max.start.pos   -0.15848
17  MA0508.1_max.start.pos  0.0939448
18  MA0597.1_max.start.pos  0.0153757
19  MA0759.1_max.start.pos   0.228075


In [12]:
# compute AUC of logistic regression model
from sklearn.metrics import roc_curve, auc, roc_auc_score
print roc_auc_score(y_test, 
                    my_logreg.predict(X_test_select))

0.601953631258


In [14]:
# plot ROC curve
roc_curve(y_test, my_logreg.predict(X_test_select))

(array([ 0.        ,  0.38645418,  1.        ]),
 array([ 0.        ,  0.59036145,  1.        ]),
 array([2, 1, 0]))

In [13]:
# calculate confusion matrix of model
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred=my_logreg.predict(X_test_select))

array([[308, 194],
       [204, 294]])

In [16]:
# return classification report
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred=my_logreg.predict(X_test_select))
print(report)

             precision    recall  f1-score   support

          0       0.60      0.61      0.61       502
          1       0.60      0.59      0.60       498

avg / total       0.60      0.60      0.60      1000



For the logistic regression model, precision is 0.60, recall is 0.59, and F1 is 0.60.  Thus the error is balanced equally among both classes.

In [85]:
# save train and test data for web app input
X_select.to_csv("top20features_X_train.csv")
X_test_select.to_csv("top20features_X_test.csv")

y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")

In [16]:
# predict test data with logistic regression using subset of features: 11 n-gram features
# subset to n-grams:
X_select2 = X_select.iloc[:, :11]
X_test_select2 = X_test_select.iloc[:, :11]

my_logreg2 = LogisticRegression()
my_logreg2.fit(X=X_select2, y=y_train)

# predict:
my_logreg2.score(X=X_test_select2, 
                y=y_test)


0.59299999999999997

## Save final model and other inputs for web app
Here I am saving the final logistic regression model as well as the means and SDs of the features in the training data for the calculations in the web app.

In [18]:
# save this basic model for webapp
import pickle

filename = 'logreg_model_basic.sav'
pickle.dump(my_logreg2, open(filename, 'wb'))

In [19]:
# also save mean and SD of all X_select rows to normalize input features
means = X_select2.mean(axis=0)
SDs = X_select2.std(axis=0)

pickle.dump(means, open("feature_means.sav", 'wb'))
pickle.dump(SDs, open("feature_SDs.sav", 'wb'))
